In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
!python --version

# Task : 
1. Create a decision tree for the dataset
2. Create confusion matrix, Gini Impurity etc
3. Perform prediction & calculate accuracy
4. Build the same program using built-in lib

# 1. Create a decision tree for the dataset

1.1 read the csv

In [ ]:
df891 = pd.read_csv('../input/titanic893/titanic_893.csv')

In [ ]:
df891.shape

In [ ]:
df891.columns

In [ ]:
df891.describe()

In [ ]:
pd.set_option("display.max_rows", None)
print(df891['Fare'].value_counts(bins = 10).sort_index())
pd.set_option("display.max_rows", 20)

In [ ]:
pd.set_option("display.max_rows", None)
print(df891['Age'].value_counts(bins = 10).sort_index())
pd.set_option("display.max_rows", 20)

# 2. Create confusion matrix, Gini Impurity etc

In [ ]:

def cal_gini(p_df, p_label, p_unique_threshold = 5):

    df_return = pd.DataFrame(columns = ['ColName', 'ColValue', 'survived', 'notSurvived', 'gini'])
    for currColName in p_df.columns:
    
        if currColName == p_label: # skip if the column is target label
            continue
    
        if p_df[currColName].nunique() > p_unique_threshold: # skip considering it root node if distinct value is more than 9
            continue
        
        df_curr_col = df891[['PassengerId', currColName, 'Survived']]
        
        for currColValue in df_curr_col[currColName].unique():
                       
            survived = int(df_curr_col[(df_curr_col[currColName] == currColValue) & (df_curr_col.Survived == 1)].PassengerId.agg(['count']))
            notSurvived = int(df_curr_col[(df_curr_col[currColName] == currColValue) & (df_curr_col.Survived == 0)].PassengerId.agg(['count']))
            total = survived + notSurvived
            
            gini = None #reset 
            if total != 0:
                gini = 1 - ((survived/total)**2) - ((notSurvived/total)**2)
            
            #print(f'currColName : {currColName} currColValue : {currColValue} survived : {survived} notSurvived : {notSurvived} gini : {gini}')
            df_return = df_return.append({'ColName' : currColName, 'ColValue' : currColValue, 'survived' : survived, 'notSurvived' : notSurvived, 'gini' : gini},ignore_index = True)
            
    return df_return



        #break
            
        #survived = int(df_curr_col[(df_curr_col.Survived == 1)].PassengerId.agg(['count']))
        #notSurvived = int(df_curr_col[(df_curr_col.Survived == 0)].PassengerId.agg(['count']))
        
        
        #print(f'col_name : {col_name} \tvalue_counts : {df_curr_col[col_name].nunique()} \t df_curr_col.shape : {df_curr_col.columns} survived : {survived} notSurvived : {notSurvived} gini : {gini}')

        

In [ ]:
myRootFeature = cal_gini(p_df = df891, p_label = 'Survived', p_unique_threshold = 3)
myRootFeature.sort_values(by=['gini'])

In [ ]:
# temp code
#adult_names = titanic.loc[titanic["Age"] > 35, "Name"]
adult_names = pd.DataFrame(df891.loc[df891["Sex"] == "male", "PassengerId"])

print(type(adult_names))
print(adult_names.head())

In [ ]:
# temp code
df1 = pd.DataFrame([["X",1,1,0],
              ["Y",0,1,0],
              ["Z",0,0,0],
              ["Y",0,1,0]],columns=["Name1","Nonprofit","Business", "Education"])    

df2 = pd.DataFrame([["Y",1,1],
              ["Z",1,1],
              ['U',1,3]],columns=["Name2","Nonprofit", "Education"])   

print(df1)
print(df2)

df1 = df1.set_index('Name1')
df2 = df2.set_index('Name2')


df1.update(df2)
print(df1)

In [ ]:
df_root = pd.DataFrame(df891)

#print(df_root.head(5), '\n\n')

df_root['branch1_SexEqMale'] = df_root["Sex"].apply(lambda x: 1 if x == 'male' else 0)
df_root['branch2_SexNEqMale'] = df_root["Sex"].apply(lambda x: 1 if x == 'female' else 0)

#df_root['branch11_PclassEq3'] = df_root["Pclass"].apply(lambda x: 1 if (x == 3) else 0)
df_root['branch11_PclassEq3'] = df_root.apply(lambda x: 1 if x['branch1_SexEqMale'] == 1 and x['Pclass'] == 3 else 0, axis = 1)
df_root['branch12_PclassNEq3'] = df_root.apply(lambda x: 1 if x['branch1_SexEqMale'] == 1 and x['Pclass'] != 3 else 0, axis = 1)

df_root['branch111_EmbarkedEqS'] = df_root.apply(lambda x: 1 if x['branch11_PclassEq3'] == 1 and x['Embarked'] == 'S' else 0, axis = 1)
df_root['branch112_EmbarkedNEqS'] = df_root.apply(lambda x: 1 if x['branch11_PclassEq3'] == 1 and x['Embarked'] != 'S' else 0, axis = 1)

for col_name in df_root.filter(regex='^branch',axis=1).columns: 
    col_count = int(df_root[(df_root[col_name] == 1)].PassengerId.agg(['count']))
    survived = int(df_root[(df_root[col_name] == 1) & (df_root.Survived == 1)].PassengerId.agg(['count']))
    notSurvived = int(df_root[(df_root[col_name] == 1) & (df_root.Survived == 0)].PassengerId.agg(['count']))
    print(f'col_name : {col_name} col_count : {col_count} survived : {survived} notSurvived : {notSurvived} Percent : {(notSurvived/(survived + notSurvived))*100}')
    



#survived = int(df_curr_col[(df_curr_col[currColName] == currColValue) & (df_curr_col.Survived == 1)].PassengerId.agg(['count']))

#print(df_root.head(5), '\n\n')

#df_branch1_SexEqMale = df_root[df_root['Sex'] == 'male']
#df_branch2_SexEqFemale = df_root[df_root['Sex'] == 'female']

#df_branch1_SexEqMale = df_root.loc[df_root['Sex'] == 'male', 'PassengerId']
#df_branch1_SexEqFeMale = df_root.loc[df_root['Sex'] == 'female', 'PassengerId']

#df_branch11_PclassEq3 = df_branch1_SexEqMale[df_branch1_SexEqMale['Pclass'] == 3]
#df_branch11_PclassNEq3 = df_branch1_SexEqMale[df_branch1_SexEqMale['Pclass'] != 3]

#df_branch111_EmbarkedEqS = df_branch11_PclassEq3[df_branch11_PclassEq3['Embarked'] == 'S']
#df_branch111_EmbarkedNEqS = df_branch11_PclassEq3[df_branch11_PclassEq3['Embarked'] != 'S']

# 3. Perform prediction & calculate accuracy

Performing prediction for Male passenger only

In [ ]:
for i in range(1,6):
    print (i)

In [ ]:
# our analysis says that Passenger = Male & Pclass = 3 has 86% chances of not surviving

# for our prediction, ,we will discard any row which is not male

for i in range(0,10):
    
    df_prediction = None
    correctPredCount = None
    inCorrectPredCount = None
    accuracy = None
    maxPassengerId = None
    minPassengerId = None
    
    r = random.randint(70, 100)
    
    df_test = df_root[(df_root['Sex'] == 'male')].sample(n = r)

    df_prediction = df_test[['PassengerId', 'Sex', 'Pclass', 'Survived']]

    df_prediction['Survived_Pred'] = df_prediction.apply(lambda x: 0 if x['Sex'] == 'male' and x['Pclass'] == 3 else 1, axis = 1)
    df_prediction['Pred_Match'] = df_prediction.apply(lambda x: 1 if x['Survived'] == x['Survived_Pred'] else 0, axis = 1)

    maxPassengerId = df_prediction['PassengerId'].max()
    minPassengerId = df_prediction['PassengerId'].min()
    correctPredCount = df_prediction[(df_prediction['Pred_Match'] == 1)].PassengerId.count()
    inCorrectPredCount = df_prediction[(df_prediction['Pred_Match'] == 0)].PassengerId.count()

    accuracy = int(((correctPredCount/(correctPredCount + inCorrectPredCount))) * 100)

    print(f'Iteration : {i}. Sample Size : {r} MaxPassengerId : {maxPassengerId} MinPassengerId : {minPassengerId} \tMy Decision Tress Accuracy : {accuracy} %. ')




In [ ]:
import random 

r1 = random.randint(1, 891)
print("Random number between 0 and 10 is % s" % (r1))

In [ ]:
# delete me
df1 = pd.DataFrame({'PID': range(1, 6), 'Sex': ['F', 'M', 'M', 'F', 'M']})

print(df1, '\n')

#df_amalapuram['Candidate Won']=df_amalapuram['% of Votes'].apply(lambda x:"Won" if x==df_amalapuram['% of Votes'].max() else "Loss")
#df_andhrapradesh['Candidate Won']=df_andhrapradesh.apply(lambda row:update_candidateresult(df_andhrapradesh,row['Constituency'],row['% of Votes']),axis=1)
# df_branch1_SexEqMale = df_root[df_root['Sex'] == 'male']

#df1 = df1.assign(Bucket1 = lambda x: (x.PID + 1))
#df1 = pd.DataFrame(df1['Sex']).assign(Bucket1 = lambda x: 'Y' if x == 'M' else 'N')

df1['Bucket1'] = df1["Sex"].apply(lambda x: 'Y' if x == 'M' else 'N')

#df = df["one"].apply(lambda x: x*10 if x<2 else x*100)

print(df1)

In [ ]:
#delete me
df=pd.DataFrame({"one":[1,2,3,4,5],"two":[6,7,8,9,10]})

print(df, '\n\n')

df = df["one"].apply(lambda x: 'L' if x<2 else 'M')

print(df)

In [ ]:
df891.columns

In [ ]:
df_categorized = None
df_categorized = pd.DataFrame(df891)

#df_categorized['df_branch1_SexEqMale'] = None
#df_categorized['df_branch2_SexEqFemale'] = None

#df_categorized['df_branch11_PclassEq3'] = None
#df_categorized['df_branch11_PclassNEq3'] = None

#df_categorized['df_branch111_EmbarkedEqS'] = None
#df_categorized['df_branch111_EmbarkedNEqS'] = None

In [ ]:
df_categorized.head()

In [ ]:
df_categorized.merge(df_branch1_SexEqMale, on = 'PassengerId', how = 'left')

In [ ]:
df_categorized.columns

In [ ]:
#df891[(df891['Sex'] == 'male')].head(20)

col_name = 'Sex'

df891[(df891[col_name] == 'male')].head(20)

In [ ]:
df_temp = df891[['PassengerId', 'Pclass', 'Survived']]

In [ ]:
for i in df_temp['Pclass'].unique():
    print(i)

In [ ]:
#df891.groupby(['Pclass', 'Survived']).agg(['count'])
df891[Pclass].su.value_counts()

In [ ]:
df891.head()

In [ ]:
df_col_survived = df891[['PassengerId', 'Survived']]


In [ ]:
df_col_survived.head()

In [ ]:
pd.set_option("display.max_rows", None)
print(df891['Pclass'].value_counts().sort_index())
pd.set_option("display.max_rows", 20)

In [ ]:
df891[(df891.Pclass == 1) & (df891.Survived == 0)].groupby(['Pclass', 'Survived']).agg(['count'])

In [ ]:
a = int(df891[(df891.Pclass == 1) & (df891.Survived == 0)].PassengerId.agg(['count']))

In [ ]:
 df_currCol = pd.DataFrame(df891['Pclass'].value_counts().sort_index())

In [ ]:
df_currCol

In [ ]:
df_currCol = df891.groupby(['Pclass', 'Survived'], as_index=False).agg({'PassengerId':['count']})

In [ ]:
df_currCol.columns

In [ ]:
for date in df.index.get_level_values('date').unique():
    print(df.loc[date])